In [115]:
from _Setting import StockSetting
import tushare  as ts
import numpy as np
import pandas as pd
from pyspark.sql import SparkSession
import matplotlib.pyplot as plt
import json
from pyspark.sql.types import StructType, StructField, StringType, DoubleType

2022-04-12 15:53:02,356 ERROR broadcast.TorrentBroadcast: Store broadcast broadcast_11146 fail, remove all pieces of the broadcast


In [56]:
spark = SparkSession.builder \
        .appName("_Spark_T01") \
        .master("yarn") \
        .config('spark.submit.pyFiles', 'file:///work/dev/stock/_Setting.py') \
        .getOrCreate()
sc = spark.sparkContext
settings = StockSetting() 
pro = ts.pro_api(settings.tushareKey)

In [72]:
      
def getdailydata(item):
    dailypath = settings.datasource_daily_path+item.ts_code
    daily_data = spark.read.format("json").load(dailypath)
    return daily_data

def getdaily_basicdata(item):
    dailypath = settings.datasource_daily_basic_path+item.ts_code
    daily_data = spark.read.format("json").load(dailypath)
    return daily_data

def getmoneyflowdata(item):
    dailypath = settings.datasource_moneyflow_path+item.ts_code
    daily_data = spark.read.format("json").load(dailypath)
    return daily_data

In [113]:
savepath = settings.datasource_path+"stock_basic_main"

data_base_main = spark.read.format("json").load(savepath)


daily_data = None
daily_basic_data = None
daily_moneyflow_data = None
x = data_base_main.collect()
index = 0
for item in x:
    index = index+1
    if(index< 1000 ):
        if(index % 50 == 49 ):
            print(index)
        try:
            return_value = getdailydata(item)
            if(daily_data == None):
                daily_data = return_value
            else :
                daily_data = daily_data.unionAll(return_value)

            return_value = getdaily_basicdata(item)
            if('dv_ratio' in return_value.columns):
                return_value = return_value.drop('dv_ratio')
            if('dv_ttm' in return_value.columns):
                return_value = return_value.drop('dv_ttm')
            if(daily_basic_data == None): 
                daily_basic_data = return_value
            else :
                daily_basic_data = daily_basic_data.unionAll(return_value)

            return_value = getmoneyflowdata(item)
            if(daily_moneyflow_data == None):
                daily_moneyflow_data = return_value
            else :
                daily_moneyflow_data = daily_moneyflow_data.unionAll(return_value)
        except Exception as ex:
            print(ex)

print("Finished")


49
99
149
199
249
299


349


399


449


499


2022-04-12 15:48:48,720 - root - ERROR - KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/work/app/anaconda3/envs/pyspark/lib/python3.8/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/work/app/anaconda3/envs/pyspark/lib/python3.8/site-packages/py4j/clientserver.py", line 475, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/work/app/anaconda3/envs/pyspark/lib/python3.8/socket.py", line 669, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [114]:
daily_moneyflow_data.show()

2022-04-12 15:50:33,204 - root - ERROR - KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/work/app/anaconda3/envs/pyspark/lib/python3.8/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/work/app/anaconda3/envs/pyspark/lib/python3.8/site-packages/py4j/clientserver.py", line 475, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/work/app/anaconda3/envs/pyspark/lib/python3.8/socket.py", line 669, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [69]:
savepath = settings.datasource_path+"stock_daily_all"

daily_data_all.write.mode("overwrite").format("json").save(savepath)


6
